In [1]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB
from graph_tool import Graph, EdgePropertyMap
from graph_tool.generation import complete_graph
from graph_tool.flow import boykov_kolmogorov_max_flow, min_st_cut #min_st_cut gives the dual sol. for x*
from graph_tool.topology import label_components

Just as in the Integer program, this class TSPInstance can be easily modified to tweak the costs for each arc and try different settings. 

The one found now, with 9 vertices, has a standard cost 10 for all edges except for two subsets [(1,2,3) and (4,5,6)] that have negative costs. With no SECS, the optimal path would be a direct edge source (0) to sink (8), and two subtours forming, one for each subset, plus an additional isolated node (7).  

This example shows how our code works with multiple subtours and isolated nodes combined. However, costs can be further personalised or totally eliminated to get a random instance, for deeper studying. 

Along the code there are references to Markdowns, where some parts are explained in more detail

In [400]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    cost: Dict[Arc, float] #contains y coordinates for all our vertices. Vertex i is formed by x[i], y[i]   

    def __init__(self, n, costs:List[float]):
        self.n = n 
        #PERSONALIZED INSTANCE       
        self.cost = {
            (i, j): 10
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }
        self.cost[(1,2)]=-1
        self.cost[(2,3)]=-1
        self.cost[(3,1)]=-1
        self.cost[(4,5)]=-1
        self.cost[(5,6)]=-1
        self.cost[(6,4)]=-1 

        #RANDOM INSTANCE
        """
        Branch-and-Cutself.cost = {}
        cost_index = 0
        for i in self.vertices():
            for j in self.vertices():
                if i != j:
                    self.cost[(i, j)] = costs[cost_index]
                    cost_index += 1
        """

    def vertices(self) -> Iterable[Vertex]: 
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:        
        costs= [uniform(-10,10) for _ in range(n*(n-1))] #RANDOM COSTS IN CASE A RANDOM INSTANCE WANTS TO BE TRIED
        return TSPIntance(n, costs=costs)

In [401]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [402]:
class BranchAndCutFractionalSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict
    G: Graph
    capacity: EdgePropertyMap

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x') 
        self.__build_model()
        self.__build_graph()

    def __build_model(self) -> None:
        self.m.addConstr(self.x.sum(0, '*') == 1) # only one outgoing  arc from source.
        self.m.addConstr(self.x.sum('*', self.tsp.n-1) == 1 ) # only one incoming  arc to sink.
        self.m.addConstrs(self.x.sum(i, '*') == self.x.sum('*',i) for i in range(1, self.tsp.n-1)) # flow conservation for all nodes
        self.m.addConstr(self.x.sum(self.tsp.n-1, '*') == 0 ) # no outgoing arcs from sink
        self.m.addConstr(self.x.sum('*', 0) == 0 ) #no incoming arcs to source
        self.m.addConstrs((self.x.sum(i, '*') <=1 for i in self.tsp.vertices())) #one outgoing arc at most
    
    def __build_graph(self) -> None: 
        self.G = complete_graph(N=self.tsp.n, self_loops=False, directed=True) #creates graph with arcs between every pair of nodes
        self.capacity = self.G.new_edge_property(value_type='double')
        self.G.edge_properties['capacity'] = self.capacity #we add a key to our dictionary with the capacity

    
    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        self.m.optimize(lambda _, where: self.__separate(where=where)) #same as integer 

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None: 
        #We only want to separate when we are in a MIPNODE optimal, or a MIPSOL
        #MIPNODE is a state in which we are checking for a sol in the tree.
        #MIPSOL means gurobi found a possibly feasible solution.
        if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
            return        
        # In MIPNODE, we must ensure that we have the optimal solution to the
        # linear relaxation at that node.        
        if where == GRB.Callback.MIPNODE and self.m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
            return
                 
        self.__update_capacity() #updates all edges capacities' to their Xij value found in solution.

        source = self.G.vertex(0) #Define a "source" vertex, which we defined to be 0.
        remaining = {node for node in range(1, self.tsp.n - 1) if node in self.G.vertices()} #set of vertices to be checked.
        
        #CHECK MARKDOWN 1. FOR DETAILS ON SEPARATION ROUTINE
        while len(remaining)>0:
            sink = self.G.vertex(remaining.pop())
            residual = boykov_kolmogorov_max_flow(
                g=self.G, source=source, target=sink,
                capacity=self.capacity
            ) #Residual is a grph copy with the residual capacities from the max flow as the edges capacities
            
            flow = residual.copy()
            flow.a = self.capacity.a - residual.a #This are the flows used in Max Flow

            # Calculate outgoing capacity of sink node
            outgoing_capacity = sum(self.capacity[e] for e in sink.out_edges())

            #total Max Flow is the sum of all flows incoming to the sink
            max_flow = sum(flow[a] for a in sink.in_edges()) 
        
            if max_flow < outgoing_capacity: 
                print(f"Found a subset which violate a SEC (flow to {sink} = {max_flow:.3f} < {outgoing_capacity:.3f})")
                
                #Once we find a violated sec, we run a min cut and the T subset is our subtour.
                
                cut = min_st_cut(g=self.G, source=source, capacity=self.capacity, residual=residual)

                assert cut[source] == True
                assert cut[sink] == False

                subtour = [j for j in self.tsp.vertices() if cut[self.G.vertex(j)] == False] 

                assert len(subtour) < self.tsp.n #we make sure this is an actual subtour (not a tour)

                #we add elimination constraint 
                self.__add_sec_for(subtour)

                remaining -=set(subtour)
            else:
                remaining -={sink}               
                
    # All edge capacities are updated to the Xij values of the Gruobi solution.     
    def __update_capacity(self) -> None:
        for e in self.G.edges():
            i, j = e.source(), e.target()

            try: # If we are in MIPSOL
                xval = self.m.cbGetSolution(self.x[i,j])
            except: # If we are in MIPNODE
                xval = self.m.cbGetNodeRel(self.x[i,j]) 

            self.capacity[e] = xval 

        #CHECK MARKDOWN 2. FOR EXPLANATION ON ISOLATED NODES
        isolated_nodes = [
            v for v in self.G.vertices()
            if sum(self.capacity[e] for e in v.out_edges()) + sum(self.capacity[e] for e in v.in_edges()) < 0.3
        ]              
        print("Isolated nodes:", [self.G.vertex_index[i] for i in isolated_nodes ])
        for node in isolated_nodes:
            edge_from_source = self.G.edge(0, node)  # Get the edge from node 0 to the isolated node
            if edge_from_source is not None:  # Ensure the edge exists
                self.capacity[edge_from_source] = 2  # Set the capacity to 2
                #print(f"Updated capacity for edge (0, {node}): {self.capacity[edge_from_source]}")     

    #Same as Integer.          
    def __tour_starting_at(self, i) -> Tour: 
        tour = [i] 
        current = self.__next_vertex(i=i)

        if current is None: #for vertices that are not in optimal sol, we will find no next vertex.
            return tour          
            
        while current != i : #esto acaba con el sink o con i 
            tour.append(current)
            if current == self.tsp.n-1:
                break
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex:
        for j in self.tsp.vertices():
            if j == i:
                continue

            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j])
            except GurobiError:
                # When optimisation is over
                x = self.x[i,j].X

            if x > 0.5:
                return j
            
        return None 
    
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j  in subtour                  
            ) <= len(subtour)-1
        )   

## 1. Separation Routine

Our separation routine is based on the following statement:

For any given node except for the source and the sink,in a graph where capacities are the potential solution to the problem,  the Max Flow computed between the source and such node must not be less than the nodes outgoing capacity. In cases where this SEC is violated, the node receives some flow which does not come from the source, but from the closed subtour. 

Therefore, for all mentioned nodes, the Max Flow from s to i is calculated and compared to i's outgoing capacity (calculated as the sum of all out_edges capacities). In cases where our SEC is violated, the Min Cut partition is calculated, and all nodes in the T side are those that form a subtour. 

## 2. Isolated nodes

To avoid including isolated nodes in subtours, we have tried to eliminate them from the graph once the capacity is updated. However, this implies that the vertices indices get updated and therefore we lose track of which nodes are to be included in the subtour, as what used to be nodes [1,3,4] could now be [1,2,3] The proposed alternative is as follows:

Once the graph capacities are updated, we re-update the edges that go from the source to each isolated node and set them to a value of 2. In this way, we make sure that the min cut will not cut through such edges. Therefore, all isolated nodes will be part of the S subset in our min cut partition, meaning only nodes in the subtour are included in T.

In [403]:
tsp = TSPIntance.random(n=9)
solver = BranchAndCutFractionalSolver(tsp=tsp)
solution = solver.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.5 LTS")

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
LazyConstraints  1

Academic license 2594094 - for non-commercial use only - registered to al___@bse.eu
Optimize a model with 20 rows, 72 columns and 216 nonzeros
Model fingerprint: 0xfddfafac
Variable types: 0 continuous, 72 integer (72 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Isolated nodes: [1, 2, 3, 4, 5, 6, 7]
Found heuristic solution: objective 10.0000000
Presolve removed 4 rows and 15 columns
Presolve time: 0.01s
Presolved: 16 rows, 57 columns, 163 nonzeros
Variable types: 0 continuous, 57 integer (57 binary)

Root relaxation: objective 4.000000e+00

In [404]:
print(solution)

[0, 8] - Cost: 10.00


In [405]:
import pandas as pd

data = [{"From": i, "To": j, "Cost": cost} for (i, j), cost in tsp.cost.items()]
print(pd.DataFrame(data))

    From  To  Cost
0      0   1    10
1      0   2    10
2      0   3    10
3      0   4    10
4      0   5    10
..   ...  ..   ...
67     8   3    10
68     8   4    10
69     8   5    10
70     8   6    10
71     8   7    10

[72 rows x 3 columns]
